## ![logo](../../img/license_header_logo.png)
> **Copyright &copy; 2021 CertifAI Sdn. Bhd.**<br>
 <br>
This program is part of OSRFramework. You can redistribute it and/or modify
<br>it under the terms of the GNU Affero General Public License as published by
<br>the Free Software Foundation, either version 3 of the License, or
<br>(at your option) any later version.
<br>
<br>This program is distributed in the hope that it will be useful,
<br>but WITHOUT ANY WARRANTY; without even the implied warranty of
<br>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
<br>GNU Affero General Public License for more details.
<br>
<br>You should have received a copy of the GNU Affero General Public License
<br>along with this program.  If not, see <http://www.gnu.org/licenses/>.

# <a name="top">04 - Data Cleanup and Missing Data</a>
Authored by: Scotrraaj Gopal - scotrraaj.gopal@certifai.ai

## <a name="description">Notebook Description</a>

When you get out of the classroom and start applying your analytical skills in the real world, you'll realise that the dataset that you obtain is not really like how you had expected. Datasets go through varieties of processes before reaching your hands, causing it to end up being unstructured and rather 'unfriendly' for the data scientist. This tutorial focuses on how to clean up your data and a handful of methods in  dealing with missing data.

By the end of this tutorial, you will be able to:

1. List, rename and delete columns.
2. Detect and count `NaN` values
3. Remove rows with `NaN` values.
4. Replace `NaN` values.

## Notebook Outline
Here's the outline for this tutorial:
1. [Notebook Description](#description)
2. [Notebook Configurations](#configuration)
3. [Data Cleanup](#clean)
    - [Handling Duplicates](#duplicates)
    - [Column Formatting](#columns)
4. [Missing Data](#missing)
    - [How Would You Know?](#detect)
    - [What To Do?](#action)
5. [Summary](#summary)
6. [Reference](#reference)

## <a name="configuration">Notebook Configurations</a>

Begin by first importing the `pandas` module and the dataset from `../../Datasets/pandas/winemag-data-130k-v2.csv`. Name the `DataFrame` object `reviews`.

In [57]:
### BEGIN SOLUTION
import pandas as pd
reviews = pd.read_csv("../../Datasets/pandas/winemag-data-130k-v2.csv", index_col=0)
### END SOLUTION


reviews

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


## <a name="clean">Data Cleanup</a>

This is a pre-processing step that has to be done before beginning to analyse the dataset. Usually the pre-processing step is the most extensive and challenging one because it has to be done carefully without introducing bias.

### <a name="duplicates">Handling Duplicates</a>

It is always important to verify that the dataset that we are working with has no duplicates, so that we know for sure that we are not aggregating duplicate rows. Removing duplicates can be considered the lowest fruit to pluck when it comes to preprocessing data.

We can do this with the `.duplicated()` method. Chaining it with the `.sum()` shows the total duplicate rows.

In [58]:
### BEGIN SOLUTION
reviews.duplicated().sum()
### END SOLUTION


9983

There are 9,983 duplicate rows in our dataset. Let's use the `.drop_duplicates()` method to drop the duplicates.

> *Note: Use the* `inplace` *flag once you are sure with the operation as this would immediately affect the root dataset variable. __Be cautious when using this flag as mistakes of using the flag on a big dataset can be costly in terms of time and effort.__*

In [59]:
print(f"Shape of reviews with duplicates = {reviews.shape}")
### BEGIN SOLUTION
reviews.drop_duplicates(inplace=True)
### END SOLUTION


print(f"Shape of reviews without duplicates = {reviews.shape}")

Shape of reviews with duplicates = (129971, 13)
Shape of reviews without duplicates = (119988, 13)


### <a name="columns">Column Formatting</a>

Routinely, datasets will have unstructured column labels with some of them being a cocktail of lowercase and uppercase words, spaces and typos. In order to make our life easier when selecting data by columns, we can spend a little time on cleaning up their names.

We can access the column labels, all at once, with `.columns`.

In [60]:
### BEGIN SOLUTION
reviews.columns
### END SOLUTION


Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
       'variety', 'winery'],
      dtype='object')

We can use the `.rename()` method to rename certain columns with a `dict` style argument.

Let's rename the `points` column to `score` and `taster_twitter_handle` to `taster_twitter`.

In [61]:
### BEGIN SOLUTION
reviews.rename(columns = {'points':'score',
                         'taster_twitter_handle':'taster_twitter'},
               inplace=True)
### END SOLUTION


reviews.columns

Index(['country', 'description', 'designation', 'score', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter', 'title',
       'variety', 'winery'],
      dtype='object')

The column labels of our dataset is actually already in good shape. The best practices for formatting column labels are as follows:
1. Lowercase letters
2. No special character such as symbols and brackets
3. Spaces replaced with underscores
4. Short but descriptive

Now, what if there are columns that doesn't bring any meaning to your problem statement?

You can always free up some space and clutter in your dataset by using `.drop()` method. This is a powerful method to remove rows or colums. Use `axis` attribute to specify if you're removing a column (`axis=1`) or a row (`axis=0`)

Let's remove the `taster_twitter` column from our `DataFrame`.

In [62]:
print(f"reviews columns before dropping: {reviews.columns}")
### BEGIN SOLUTION
reviews.drop('taster_twitter', axis=1, inplace=True)
### END SOLUTION


print(f"reviews columns after dropping: {reviews.columns}")
reviews

reviews columns before dropping: Index(['country', 'description', 'designation', 'score', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter', 'title',
       'variety', 'winery'],
      dtype='object')
reviews columns after dropping: Index(['country', 'description', 'designation', 'score', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'title', 'variety', 'winery'],
      dtype='object')


,country,description,designation,score,price,province,region_1,region_2,taster_name,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...
129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


## <a name="missing">Missing Data</a>

When investigating your data, you will most inevitably come across missing or null values which are generally placeholders for non-existent information. By `pandas` default, missing values in a dataset are given the values `NaN`, short for "Not a Number". 

### <a name="detect">How Would You Know?</a>

While you can always inspect your dataset by eyeballing out every occurence of `NaN` and dealing with them, this may not be very feasible when you have thousands of rows of data.

You can check if your `DataFrame` object has `NaN` with the method `.isna()`. Chain it with `.sum()` to obtain a total of `NaN` values in each column.

In [54]:
### BEGIN SOLUTION
reviews.isna().sum()
### END SOLUTION


country              59
description           0
designation       34545
score                 0
price              8395
province             59
region_1          19560
region_2          73219
taster_name       24917
taster_twitter    29446
title                 0
variety               1
winery                0
dtype: int64

### <a name='action'>What To Do?</a>

There are a two strategies that can be done when encountered with `NaN` values. Deciding on which strategy to go with requires intimate knowledge of the dataset and its context.

1. Remove the whole row with `NaN` values.
2. Use data imputation to fill the `NaN` values with a reasonably justified value.

#### Removing the rows with `NaN` values is rather straightforward. 

Use the `.dropna()` method to return a version of the `DataFrame` without any `NaN` values. Since we still need the `NaN` values to showcase the next strategy, we will not use the `inplace` attribute. Chain the `.reset_index()` method with appropriate attributes *(wink.. wink..)* to provide a new index flow. 

In [65]:
### BEGIN SOLUTION
reviews.dropna().reset_index(drop=True)
### END SOLUTION


,country,description,designation,score,price,province,region_1,region_2,taster_name,title,variety,winery
0,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
1,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
2,US,This wine from the Geneseo district offers aro...,Signature Selection,87,22.0,California,Paso Robles,Central Coast,Matt Kettmann,Bianchi 2011 Signature Selection Merlot (Paso ...,Merlot,Bianchi
3,US,Oak and earth intermingle around robust aromas...,King Ridge Vineyard,87,69.0,California,Sonoma Coast,Sonoma,Virginie Boone,Castello di Amorosa 2011 King Ridge Vineyard P...,Pinot Noir,Castello di Amorosa
4,US,As with many of the Erath 2010 vineyard design...,Hyland,86,50.0,Oregon,McMinnville,Willamette Valley,Paul Gregutt,Erath 2010 Hyland Pinot Noir (McMinnville),Pinot Noir,Erath
...,...,...,...,...,...,...,...,...,...,...,...,...
21970,US,"This ripe, rich, almost decadently thick wine ...",Reserve,91,105.0,Washington,Walla Walla Valley (WA),Columbia Valley,Paul Gregutt,Nicholas Cole Cellars 2004 Reserve Red (Walla ...,Red Blend,Nicholas Cole Cellars
21971,US,This pure Syrah from Reininger's estate vineya...,Ash Hollow Vineyard,91,41.0,Washington,Walla Walla Valley (WA),Columbia Valley,Paul Gregutt,Reininger 2005 Ash Hollow Vineyard Syrah (Wall...,Syrah,Reininger
21972,US,Hailing from one of the more popular vineyards...,Jurassic Park Vineyard Old Vines,90,20.0,California,Santa Ynez Valley,Central Coast,Matt Kettmann,Birichino 2013 Jurassic Park Vineyard Old Vine...,Chenin Blanc,Birichino
21973,US,There's no bones about the use of oak in this ...,Barrel Fermented,90,35.0,California,Napa Valley,Napa,Virginie Boone,Flora Springs 2013 Barrel Fermented Chardonnay...,Chardonnay,Flora Springs


#### Replace `NaN` values with data imputation.

Replacing missing values is a conventional operation to keep valuable data that have `NaN` values. We can opt into this strategy when dropping every row with missing data causes the lost of a huge chunk of data. `pandas` provide a really handy method for this problem: `.fillna()`. This method allows a few different ways of replacing the values to mitigate such data.

For example, we can simply replace the `NaN` values in every row with 0.

In [74]:
### BEGIN SOLUTION
reviews.fillna(0)
### END SOLUTION


,country,description,designation,score,price,province,region_1,region_2,taster_name,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,0.0,Sicily & Sardinia,Etna,0,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,0,0,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",0,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,0,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...
129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,0,0,Anna Lee C. Iijima,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,US,Citation is given as much as a decade of bottl...,0,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,0,Roger Voss,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,France,"A dry style of Pinot Gris, this is crisp with ...",0,90,32.0,Alsace,Alsace,0,Roger Voss,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


There is also another method called `.replace()` that can be used to deal with this issue. But this method is more versatile in its uses as it can be utilized to even replace non-`NaN` values. 

For example, let's say the element in the `variety` column called `White Blend` has been updated recently to `White Mix`. We can easily implement this change in our dataset with `.replace()`.

In [78]:
print(f"First row in 'variety' column before replace: {reviews.variety[0]}")
### BEGIN SOLUTION
reviews.variety.replace('White Blend', 'White Mix', inplace=True)
### END SOLUTION


print(f"First row in 'variety' column after replace: {reviews.variety[0]}")

First row in 'variety' column before replace: White Blend
First row in 'variety' column after replace: White Mix


We may use `.replace()` to replace `NaN` values with values that can be more relatable to the data. For example, `NaN` values in the `price` column can be replaced with `Free` or the average price etc.

##  <a name="summary">Summary</a>
To conclude, you should now be able to:

1. List, rename and delete columns.
2. Detect and count `NaN` values
3. Remove rows with `NaN` values.
4. Replace `NaN` values.

Congratulations, that concludes this tutorial. Exploring, cleaning and transforming data is an essential skill in data science. After some practice, you should be really comfortable with most of the basics. 

In your learning journey, you may come across errors in many different forms. Don't let that discourage you as even the best programmers have them too. Steer through the error by interpreting it and try your level best to debug your code. You may also use this [guide](https://geo-python.github.io/site/notebooks/L6/errors.html) as a point of reference.

There is an exercise notebook in this folder that can be helpful for you to test out everything that you've learnt. Don't forget to check it out!

**See you and happy coding!**

## <a name="reference">Reference</a>
* [Dataset Source](https://www.kaggle.com/zynicide/wine-reviews)

<font size=2>[Back to Top](#top)</font>